<a href="https://colab.research.google.com/drive/1sNax0Uq-gPWO0SZgTEEYPrdBTRa9VAPA?usp=sharing" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Entrenamiento para NMT con mecanísmos de atención

Este código corresponde al entrenamiento de una NMT con mecanismo de atención. Se sugiere el uso de colab para correr este notebook y tener un mejor performance.

In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time

/home/francisco/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [0]:
# Download the file
str_RutaZip = tf.keras.utils.get_file(
    'spa-eng.zip', origin='http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip',
    extract=True)

str_RutaArchivo = os.path.dirname(str_RutaZip)+"/spa-eng/spa.txt"

In [0]:
# Converts the unicode file to ascii
def ConvertirUnicodeToAscii(str_Texto):
  return ''.join(c for c in unicodedata.normalize('NFD', str_Texto) if unicodedata.category(c) != 'Mn')


def PrepararOraciones(str_Oracion):
  str_Oracion = ConvertirUnicodeToAscii(str_Oracion.lower().strip())

  # Se crea un espacio entre las palabras y signos de puntuación
  str_Oracion = re.sub(r"([?.!,¿])", r" \1 ", str_Oracion)
  str_Oracion = re.sub(r'[" "]+', " ", str_Oracion)

  # Se reemplaza todo por un espacio excepto letras y signos especificados
  str_Oracion = re.sub(r"[^a-zA-Z?.!,¿]+", " ", str_Oracion)

  str_Oracion = str_Oracion.strip()

  # Se agrega el token de inicio y fin
  str_Oracion = '<start> ' + str_Oracion + ' <end>'
  return str_Oracion

In [4]:
str_Oracion_Target = u"May I borrow this book?"
str_Oracion_Input = u"¿Puedo tomar prestado este libro?"

print(PrepararOraciones(str_Oracion_Target))
print(PrepararOraciones(str_Oracion_Input).encode('utf-8'))

<start> may i borrow this book ? <end>
b'<start> \xc2\xbf puedo tomar prestado este libro ? <end>'


In [0]:
# 1. Quitar acentos
# 2. Limpiar oraciones
# 3. Regresar parejas de textos en el formtato: [Target, Input]
def PrepararDataset(str_RutaArchivo, nbr_CantEjemplos):

  str_Lineas = io.open(str_RutaArchivo, encoding='UTF-8').read().strip().split('\n')

  list_ParejaDeFrases = [[PrepararOraciones(str_Oracion) for str_Oracion in str_Linea.split('\t')]  for str_Linea in str_Lineas[:nbr_CantEjemplos]]
  
  return zip(*list_ParejaDeFrases)

In [6]:
tupl_Target, tupl_Input = PrepararDataset(str_RutaArchivo, None)

# Mostramos la última frase de cada idioma
print(tupl_Target[-1])
print(tupl_Input[-1])

<start> if you want to sound like a native speaker , you must be willing to practice saying the same sentence over and over in the same way that banjo players practice the same phrase over and over until they can play it correctly and at the desired tempo . <end>
<start> si quieres sonar como un hablante nativo , debes estar dispuesto a practicar diciendo la misma frase una y otra vez de la misma manera en que un musico de banjo practica el mismo fraseo una y otra vez hasta que lo puedan tocar correctamente y en el tiempo esperado . <end>


In [0]:
def Tokenizar(tupl_Frases):
  
  Tokenizador_Frases = tf.keras.preprocessing.text.Tokenizer(filters='')
  Tokenizador_Frases.fit_on_texts(tupl_Frases)

  tensor = Tokenizador_Frases.texts_to_sequences(tupl_Frases)

  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')

  return tensor, Tokenizador_Frases
  

In [0]:
def CargarDataset(str_RutaArchivo, nbr_CantEjemplos=None):
  
  tupl_Target, tupl_Input = PrepararDataset(str_RutaArchivo, nbr_CantEjemplos)

  # Idioma Input
  tensor_Input, Tokenizador_Frases_Input = Tokenizar(tupl_Input)
  
  # Idioma Target
  tensor_Target, Tokenizador_Frases_Target = Tokenizar(tupl_Target)

  return tensor_Input, tensor_Target, Tokenizador_Frases_Input, Tokenizador_Frases_Target

In [0]:
# nbr_CantEjemplos = 100
nbr_CantEjemplos = 100000 # 50,000

tensor_Input, tensor_Target, Tokenizador_Frases_Input, Tokenizador_Frases_Target = CargarDataset(str_RutaArchivo, nbr_CantEjemplos)

# Obtenemos el tamanio máximo de los tensores
nbr_TamanioMax_FraseTarget, nbr_TamanioMax_FraseInput = tensor_Target.shape[1], tensor_Input.shape[1]

In [10]:
# Separamos train y test 80-20
tensor_Input_train, tensor_Input_val, tensor_Target_train, tensor_Target_val = train_test_split(tensor_Input, tensor_Target, test_size=0.2)

# Show length
print(len(tensor_Input_train), len(tensor_Target_train), len(tensor_Input_val), len(tensor_Target_val))


80000 80000 20000 20000


In [0]:
def convert(Tokenizador, tensor):
  for t in tensor:
    if t!=0:
      print ("%d ----> %s" % (t, Tokenizador.index_word[t]))

In [12]:
print ("Input Language; index to word mapping")
convert(Tokenizador_Frases_Input, tensor_Input_train[0])
print ()
print ("Target Language; index to word mapping")
convert(Tokenizador_Frases_Target, tensor_Target_train[0])

Input Language; index to word mapping
1 ----> <start>
13 ----> es
1894 ----> inutil
4694 ----> hablarle
72 ----> cuando
18 ----> esta
1057 ----> enojado
3 ----> .
2 ----> <end>

Target Language; index to word mapping
1 ----> <start>
48 ----> there
15 ----> s
71 ----> no
333 ----> talking
7 ----> to
46 ----> him
91 ----> when
13 ----> he
15 ----> s
327 ----> angry
3 ----> .
2 ----> <end>


In [0]:
nbr_TamanioBuffer = len(tensor_Input_train)
# nbr_TamanioBatch = 16
nbr_TamanioBatch = 64
nbr_steps_per_epoch = len(tensor_Input_train)//nbr_TamanioBatch
nbr_EmbeddingDim = 256
# nbr_EncDec_Units = 10
nbr_EncDec_Units = 1024
nbr_TamanioVoc_Input = len(Tokenizador_Frases_Input.word_index)+1
nbr_TamanioVoc_Target = len(Tokenizador_Frases_Target.word_index)+1

dataset = tf.data.Dataset.from_tensor_slices((tensor_Input_train, tensor_Target_train)).shuffle(nbr_TamanioBuffer)
dataset = dataset.batch(nbr_TamanioBatch, drop_remainder=True)

In [14]:
tensor_Input_Batch_Ej, tensor_Target_Batch_Ej = next(iter(dataset))

tensor_Input_Batch_Ej.shape, tensor_Target_Batch_Ej.shape

(TensorShape([64, 20]), TensorShape([64, 17]))

In [0]:
class Encoder(tf.keras.Model):
  def __init__(self, nbr_TamanioVoc_Input, nbr_EmbeddingDim, nbr_EncDec_Units, nbr_TamanioBatch):
    super(Encoder, self).__init__()
    self.nbr_TamanioBatch = nbr_TamanioBatch
    self.nbr_EncDec_Units = nbr_EncDec_Units
    self.embedding = tf.keras.layers.Embedding(nbr_TamanioVoc_Input, nbr_EmbeddingDim)
    self.gru = tf.keras.layers.GRU(self.nbr_EncDec_Units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')

  def call(self, tensor_Input_Batch, tensor_Encoder_Oculto):
    
    tensor_Input_Batch_Embedding = self.embedding(tensor_Input_Batch)
    
    tensor_Encoder_Output, tensor_Encoder_Estado_Oculto = self.gru(tensor_Input_Batch_Embedding, initial_state = tensor_Encoder_Oculto)
    return tensor_Encoder_Output, tensor_Encoder_Estado_Oculto

  def initialize_hidden_state(self):
    return tf.zeros((self.nbr_TamanioBatch, self.nbr_EncDec_Units))

In [16]:
encoder = Encoder(nbr_TamanioVoc_Input, nbr_EmbeddingDim, nbr_EncDec_Units, nbr_TamanioBatch)

# sample input
tensor_Encoder_Estado_Oculto_Init = encoder.initialize_hidden_state()

tensor_Encoder_Output, tensor_Encoder_Estado_Oculto = encoder.call(tensor_Input_Batch_Ej, tensor_Encoder_Estado_Oculto_Init)
print ('--Encoder output shape: (batch size, sequence length, units) {}'.format(tensor_Encoder_Output.shape))
print ('--Encoder Hidden state shape: (batch size, units) {}'.format(tensor_Encoder_Estado_Oculto.shape))

--Encoder output shape: (batch size, sequence length, units) (64, 20, 1024)
--Encoder Hidden state shape: (batch size, units) (64, 1024)


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, tensor_Oculto, tensor_Encoder_Output):
    # tensor_Oculto hidden state shape == (batch_size, hidden size)
    # tensor_Oculto_with_time_axis shape == (batch_size, 1, hidden size)
    # tensor_Encoder_Output shape == (batch_size, max_len, hidden size)
    # we are doing this to broadcast addition along the time axis to calculate the score
    tensor_Oculto_with_time_axis = tf.expand_dims(tensor_Oculto, 1)
    # print('tensor_Oculto: ', tensor_Oculto) # MJMA
    # print('tensor_Oculto_with_time_axis: ', tensor_Oculto_with_time_axis) # MJMA

    # score shape == (batch_size, max_length, 1)
    # we get 1 at the last axis because we are applying score to self.V
    # the shape of the tensor before applying self.V is (batch_size, max_length, units)
    score = self.V(tf.nn.tanh(
                              self.W1(tensor_Oculto_with_time_axis) + self.W2(tensor_Encoder_Output)
                              )
                  )

    # tensor_attention_weights shape == (batch_size, max_length, 1)
    tensor_attention_weights = tf.nn.softmax(score, axis=1)

    # tensor_context_vector shape after sum == (batch_size, hidden_size)
    tensor_context_vector = tensor_attention_weights * tensor_Encoder_Output
    tensor_context_vector = tf.reduce_sum(tensor_context_vector, axis=1)

    return tensor_context_vector, tensor_attention_weights

In [18]:
attention_layer = BahdanauAttention(10)
tensor_context_vector, tensor_attention_weights = attention_layer.call(tensor_Encoder_Estado_Oculto_Init, tensor_Encoder_Output)

print("Attention result shape: (batch size, units) {}".format(tensor_context_vector.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(tensor_attention_weights.shape))

Attention result shape: (batch size, units) (64, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (64, 20, 1)


In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, nbr_EmbeddingDim, nbr_EncDec_Units, nbr_TamanioBatch):
    super(Decoder, self).__init__()
    self.nbr_TamanioBatch = nbr_TamanioBatch
    self.nbr_EncDec_Units = nbr_EncDec_Units
    self.embedding = tf.keras.layers.Embedding(vocab_size, nbr_EmbeddingDim)
    self.gru = tf.keras.layers.GRU(self.nbr_EncDec_Units,
                                   return_sequences=True,
                                   return_state=True,
                                   recurrent_initializer='glorot_uniform')
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.nbr_EncDec_Units)

  def call(self, tensor_input, tensor_Oculto, tensor_Encoder_Output):
    
    # tensor_Encoder_Output shape == (batch_size, max_length, hidden_size)
    tensor_context_vector, tensor_attention_weights = self.attention.call(tensor_Oculto, tensor_Encoder_Output)
    

    # tensor_input shape after passing through embedding == (batch_size, 1, nbr_EmbeddingDim)
    tensor_input_embedding = self.embedding(tensor_input)

    # x shape after concatenation == (batch_size, 1, nbr_EmbeddingDim + hidden_size)
    tensor_input_embedding_concatenado = tf.concat([tf.expand_dims(tensor_context_vector, 1), tensor_input_embedding], axis=-1)

    # passing the concatenated vector to the GRU
    tensor_output, tensor_Decoder_Estado = self.gru(tensor_input_embedding_concatenado)

    # tensor_output shape == (batch_size * 1, hidden_size)
    tensor_output = tf.reshape(tensor_output, (-1, tensor_output.shape[2]))

    # tensor_output shape == (batch_size, vocab)
    tensor_Decoder_Output = self.fc(tensor_output)

    return tensor_Decoder_Output, tensor_Decoder_Estado, tensor_attention_weights

In [20]:
decoder = Decoder(nbr_TamanioVoc_Target, nbr_EmbeddingDim, nbr_EncDec_Units, nbr_TamanioBatch)
tensor_aleatorio_batch = tf.random.uniform((nbr_TamanioBatch, 1))
tensor_Decoder_Output, _, _ = decoder.call(tensor_aleatorio_batch,
                                      tensor_Encoder_Estado_Oculto_Init,
                                      tensor_Encoder_Output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(tensor_Decoder_Output.shape))

Decoder output shape: (batch_size, vocab size) (64, 10785)


## Define the optimizer and the loss function

In [0]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [0]:
@tf.function
def train_step(tensor_Input_Batch, tensor_Target_Batch, tensor_Encoder_Oculto_Init):
  loss = 0

  with tf.GradientTape() as tape:

    tensor_Encoder_Output, tensor_Encoder_Estado_Oculto = encoder.call(tensor_Input_Batch, tensor_Encoder_Oculto_Init)

    tensor_Decoder_Estado = tensor_Encoder_Estado_Oculto

    tensor_decoder_input = tf.expand_dims([Tokenizador_Frases_Target.word_index['<start>']] * nbr_TamanioBatch, 1)

    # Teacher forcing - feeding the target as the next input
    for token in range(1, tensor_Target_Batch.shape[1]):
      # passing tensor_Encoder_Output to the decoder
      
      # tensor_Decoder_Output, tensor_Decoder_Estado, tensor_attention_weights
      tensor_Predicciones, tensor_Decoder_Estado, _ = decoder.call(tensor_decoder_input, tensor_Decoder_Estado, tensor_Encoder_Output)

      loss += loss_function(tensor_Target_Batch[:, token], tensor_Predicciones)

      # using teacher forcing
      tensor_decoder_input = tf.expand_dims(tensor_Target_Batch[:, token], 1)
      

  batch_loss = (loss / int(tensor_Target_Batch.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [0]:
EPOCHS = 10

for epoch in range(EPOCHS):
  start = time.time()

  tensor_Encoder_Oculto_Init = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (tensor_Input_Batch, tensor_Target_Batch)) in enumerate(dataset.take(nbr_steps_per_epoch)):
    batch_loss = train_step(tensor_Input_Batch, tensor_Target_Batch, tensor_Encoder_Oculto_Init)
    total_loss += batch_loss

    if batch % 100 == 0:
      print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / nbr_steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Epoch 1 Batch 0 Loss 4.4893
Epoch 1 Batch 100 Loss 2.3471
Epoch 1 Batch 200 Loss 2.0503
Epoch 1 Batch 300 Loss 1.8419
Epoch 1 Batch 400 Loss 1.7558
Epoch 1 Batch 500 Loss 1.7042
Epoch 1 Batch 600 Loss 1.4851
Epoch 1 Batch 700 Loss 1.6245
Epoch 1 Batch 800 Loss 1.3261
Epoch 1 Batch 900 Loss 1.3894
Epoch 1 Batch 1000 Loss 1.3711
Epoch 1 Batch 1100 Loss 1.2765
Epoch 1 Batch 1200 Loss 1.2025
Epoch 1 Loss 1.6616
Time taken for 1 epoch 458.8923394680023 sec

Epoch 2 Batch 0 Loss 1.0880
Epoch 2 Batch 100 Loss 1.1016
Epoch 2 Batch 200 Loss 1.0305
Epoch 2 Batch 300 Loss 1.1234
Epoch 2 Batch 400 Loss 0.9347
Epoch 2 Batch 500 Loss 1.0172
Epoch 2 Batch 600 Loss 0.9297
Epoch 2 Batch 700 Loss 0.9790
Epoch 2 Batch 800 Loss 0.8613
Epoch 2 Batch 900 Loss 0.7256
Epoch 2 Batch 1000 Loss 0.7411
Epoch 2 Batch 1100 Loss 0.6722
Epoch 2 Batch 1200 Loss 0.7391
Epoch 2 Loss 0.9041
Time taken for 1 epoch 436.1432101726532 sec

Epoch 3 Batch 0 Loss 0.6183
Epoch 3 Batch 100 Loss 0.5582
Epoch 3 Batch 200 Loss 0.720

In [0]:
def evaluate(str_Oracion):
  np_attention_plot = np.zeros((nbr_TamanioMax_FraseTarget, nbr_TamanioMax_FraseInput))

  str_OracionPreparada = PrepararOraciones(str_Oracion)
  
  inputs = [Tokenizador_Frases_Input.word_index[i] for i in str_OracionPreparada.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                         maxlen=nbr_TamanioMax_FraseInput,
                                                         padding='post')
  tensor_Input = tf.convert_to_tensor(inputs)

  result = ''

  tensor_Encoder_Oculto_Init = [tf.zeros((1, nbr_EncDec_Units))]

  tensor_Encoder_Output, tensor_Encoder_Estado_Oculto = encoder(tensor_Input, tensor_Encoder_Oculto_Init)

  tensor_Decoder_Estado_Oculto = tensor_Encoder_Estado_Oculto
  tensor_Decoder_Input = tf.expand_dims([Tokenizador_Frases_Target.word_index['<start>']], 0)

  # nbr_TamanioMax_FraseTarget, nbr_TamanioMax_FraseInput
  for token in range(nbr_TamanioMax_FraseTarget):
    # tensor_decoder_output, tensor_state, tensor_attention_weights
    tensor_Predicciones, tensor_Decoder_Estado_Oculto, tensor_attention_weights = decoder.call(tensor_Decoder_Input,
                                                         tensor_Decoder_Estado_Oculto,
                                                         tensor_Encoder_Output)

    # storing the attention weights to plot later on
    tensor_attention_weights = tf.reshape(tensor_attention_weights, (-1, ))
    np_attention_plot[token] = tensor_attention_weights.numpy()

    predicted_id = tf.argmax(tensor_Predicciones[0]).numpy()

    result += Tokenizador_Frases_Target.index_word[predicted_id] + ' '

    if Tokenizador_Frases_Target.index_word[predicted_id] == '<end>':
      return result, str_OracionPreparada, np_attention_plot

    # the predicted ID is fed back into the model
    tensor_Decoder_Input = tf.expand_dims([predicted_id], 0)

  return result, str_OracionPreparada, np_attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def Traducir(str_Oracion):
  result, sentence, np_attention_plot = evaluate(str_Oracion)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  np_attention_plot = np_attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  plot_attention(np_attention_plot, sentence.split(' '), result.split(' '))

## Restore the latest checkpoint and test

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
Traducir(u'hace mucho frio aqui.')

In [0]:
Traducir(u'esta es mi vida.')

In [0]:
Traducir(u'¿todavia estan en casa?')

In [0]:
# wrong translation
Traducir(u'trata de averiguarlo.')

In [0]:
# MJMA prueba
Traducir(u'¿Puedo tomar prestado este libro?')

In [0]:
# MJMA prueba
#Traducir('¿Me prestas un cuaderno?')
#Traducir('Cuidate')

#Traducir('Sé que ha sido realmente duro para ti.')
#Traducir('Le mentí a mi novia sobre mi edad.')
Traducir(u'intentaba leer tantos libros como pudiera.')


## Next steps

* [Download a different dataset](http://www.manythings.org/anki/) to experiment with translations, for example, English to German, or English to French.
* Experiment with training on a larger dataset, or using more epochs
